# Trying out something similar to the project...

In [48]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets
import numpy as np
import pandas as pd
from PIL import Image
import argparse
import sys

In [3]:
class BirdDataset(Dataset):
    def __init__(self, csv_file, transform=None):

        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.birds = pd.read_csv(csv_file, sep=' ', names=['path','class_id'])
        self.transform = transform

    def __len__(self):
        return len(self.birds)

    def __getitem__(self, idx):
        img_name = self.birds.iloc[idx, 0]
        img = Image.open(img_name)
        img = np.array(img)
        label = self.birds.iloc[idx, 1]
        sample = {'image': img, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [4]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image).type(torch.FloatTensor),
                'label': torch.tensor(int(label)).type(torch.FloatTensor)}

In [35]:
class CNN(nn.Module):
    """ based on https://medium.com/ml2vec/intro-to-pytorch-with-image-classification-on-a-fashion-clothes-dataset-e589682df0c5 """
    def __init__(self, image_size, num_classes):
        self.image_size = image_size
        self.num_classes = num_classes
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc1 = nn.Linear(self.image_size*self.image_size*2, self.image_size*2)
        self.fc2 = nn.Linear(self.image_size*2, self.num_classes)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [49]:
def build_datasets(path, batch_size):
    print("Building datasets...")
    train_csv = os.path.join(path, 'train/train_data.txt')
    test_csv = os.path.join(path, 'test/test_data.txt')
    train_dataset = BirdDataset(csv_file=train_csv, transform=ToTensor())
    test_dataset = BirdDataset(csv_file=test_csv, transform=ToTensor())
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

In [113]:
def train(model, train_loader, num_epochs, learning_rate, display_every):
    print("Training the model")

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    total_step = len(train_loader)
    loss_list = []
    acc_list = []

    for epoch in range(num_epochs):
        for i, data in enumerate(train_loader):
            # Run the forward pass
            outputs = model(data['image'])
            labels = data['label'].type(torch.LongTensor)
            loss = criterion(outputs, labels)
            loss_list.append(loss.item())

            # Backprop and perform Adam optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track the accuracy
            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()
            acc_list.append(correct / total)

            if (i + 1) % display_every == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                              (correct / total) * 100))
    return loss_list, acc_list

In [114]:
def test(model, test_loader):
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            outputs = model(data['image'])
            _, predicted = torch.max(outputs.data, 1)
            labels = data['label'].type(torch.LongTensor)
            total += labels.size(0)
            print(total + ".", end='')
            correct += (predicted == labels).sum().item()

        print('Accuracy on the test images: {} %'.format((correct / total) * 100))

In [115]:
def plot(loss_list, acc_list):
    p = figure(y_axis_label='Loss', width=850, y_range=(0, 1), title='ConvNet results')
    p.extra_y_ranges = {'Accuracy': Range1d(start=0, end=100)}
    p.add_layout(LinearAxis(y_range_name='Accuracy', axis_label='Accuracy (%)'), 'right')
    p.line(np.arange(len(loss_list)), loss_list)
    p.line(np.arange(len(loss_list)), 
           np.array(acc_list) * 100,
           y_range_name='Accuracy',
           color='red')
    show(p)

In [116]:
def get_dirs(image, output):
    # These are my directory names on my testing system (a Mac)
    # The defaults in FLAGS are my directory names on my coding system (my Windows laptop)
    # You can override them both with --image_dir and --output_dir
    IMAGE_BASE = '/Users/leaf/CS767/data128/' 
    OUTPUT_BASE = '/Users/leaf/CS767/birds/output'

    image_dir = image
    output_dir = output
    if not os.path.exists(image_dir):
        print("Not a valid image directory {}, try using --image_dir flag".format(image_dir))
        return None, None
    if not os.path.exists(output_dir):
        print("Not a valid output directory {}, try using --output_dir flag".format(output_dir))
        return None, None
    else:
        return image_dir, output_dir

In [124]:
EPOCHS = 6
DISP_EVERY = 40
LEARN_RATE = 0.001
BATCH_SIZE = 100
IMAGE_DIR = "C:/datasets/CUB_200_2011/processed/data128/"
OUT_DIR = "C:/Users/Leaf/Google Drive/School/BU-MET-CS-767/Project/birds/output/"

In [125]:
def main():
    IMAGE_SIZE = 128
    NUM_CLASSES = 200
    MODEL_FILE = 'models/nn_20181013.ckpt'
    image, output = get_dirs(IMAGE_DIR, OUT_DIR)
    if image and output:
        train_loader, test_loader = build_datasets(image, BATCH_SIZE)
        model = CNN(IMAGE_SIZE, NUM_CLASSES)
        loss_list, acc_list = train(
            model, 
            train_loader, 
            EPOCHS, 
            LEARN_RATE,
            DISP_EVERY)
        model.eval()
        test(model, test_loader)
        torch.save(model.state_dict(), os.path.join(output, MODEL_FILE))
        plot(loss_list, acc_list)

In [126]:
main()

Building datasets...
Training the model
Epoch [1/6], Step [40/120], Loss: 6.1615, Accuracy: 3.00%
Epoch [1/6], Step [80/120], Loss: 5.0720, Accuracy: 6.00%
Epoch [1/6], Step [120/120], Loss: 5.0050, Accuracy: 3.41%
Epoch [2/6], Step [40/120], Loss: 3.7170, Accuracy: 22.00%
Epoch [2/6], Step [80/120], Loss: 3.7864, Accuracy: 16.00%
Epoch [2/6], Step [120/120], Loss: 3.7766, Accuracy: 18.18%
Epoch [3/6], Step [40/120], Loss: 2.8167, Accuracy: 34.00%
Epoch [3/6], Step [80/120], Loss: 2.5496, Accuracy: 46.00%
Epoch [3/6], Step [120/120], Loss: 2.6799, Accuracy: 29.55%
Epoch [4/6], Step [40/120], Loss: 1.5895, Accuracy: 64.00%
Epoch [4/6], Step [80/120], Loss: 1.7166, Accuracy: 57.00%
Epoch [4/6], Step [120/120], Loss: 2.0763, Accuracy: 46.59%
Epoch [5/6], Step [40/120], Loss: 0.4176, Accuracy: 90.00%
Epoch [5/6], Step [80/120], Loss: 0.7564, Accuracy: 80.00%
Epoch [5/6], Step [120/120], Loss: 0.7437, Accuracy: 76.14%
Epoch [6/6], Step [40/120], Loss: 0.1836, Accuracy: 97.00%
Epoch [6/6], S

TypeError: unsupported operand type(s) for +: 'int' and 'str'

# Merging folders from different data sets..

In [205]:
import pandas as pd
import numpy as np
import os

In [90]:
birdlist = pd.read_csv("C:/datasets/Combined/classes.txt", header=0)
cub_species = list(birdlist["cub_id"])
b_species = list(birdlist["birdsnap_id"])
na_species = list(birdlist["nabirds_id"])

In [91]:
# Done!
# b_file = "C:/datasets/birdsnap/birdsnap/images.txt"
# b_cols = ["path","species_id","bb_x1","bb_y1","bb_x2","bb_y2"]
# birdsnap = pd.read_csv(b_file, delimiter='\t', header=0, usecols=b_cols)
# birdsnap = birdsnap[birdsnap["species_id"].isin(b_species)]
# birdsnap.to_csv("C:/datasets/Combined/birdsnap.txt", header=True, index=False)

In [160]:
# DONE! This is how I build the "ready" file for birdsnap
# b_file = "C:/datasets/Combined/birdsnap.txt"
# birdsnap = pd.read_csv(b_file, header=0)
# birdsnap = pd.merge(birdsnap,birdlist,on="birdsnap_id")
# birdsnap = birdsnap.drop(["cub_id","nabirds_id","folder_name","birdsnap_id"],axis=1)
# # birdsnap
# birdsnap.to_csv("C:/datasets/Combined/birdsnap_ready.txt", header=True, index=False)

In [162]:
# DONE! This is how I built the "ready" file for nabirds
# na_dir = "C:/datasets/nabirds.tar/nabirds/"
# na = pd.read_csv(na_dir + "image_class_labels.txt", delimiter=' ', header=None, names=["image_id","nabirds_id"])
# na = na[na["nabirds_id"].isin(na_species)]
# na_boxes = pd.read_csv(na_dir + "bounding_boxes.txt", delimiter=' ', header=None, names=["image_id","left","upper","width","height"])
# boxes = pd.merge(na,na_boxes,on='image_id')
# boxes['right'] = boxes['left'] + boxes['width']
# boxes['lower'] = boxes['upper'] + boxes['height']
# boxes.drop(['width','height'], axis=1, inplace=True)
# nabirds = pd.merge(boxes,birdlist,on="nabirds_id")
# nabirds["path"] = nabirds["folder_name"] + "/" + nabirds["image_id"] + ".jpg"
# # nabirds = nabirds.drop(["cub_id","birdsnap_id","folder_name","image_id","nabirds_id"],axis=1)
# nabirds = nabirds.reindex(["path","left","upper","right","lower","id"],axis=1)
# nabirds.to_csv("C:/datasets/Combined/nabirds_ready.txt", header=True, index=False)

In [186]:
# DONE! This is how I built the "ready" file for cub-200-2011
# DATA_PATH="C:/datasets/CUB_200_2011/CUB_200_2011/"
# cub = pd.read_csv(DATA_PATH + "bounding_boxes.txt",' ',names=['image_id','left','upper','width','height'])
# # Set up right and lower values, drop the width and height
# cub['right'] = cub['left'] + cub['width']
# cub['lower'] = cub['upper'] + cub['height']
# cub.drop(['width','height'], axis=1, inplace=True)
# # Read in the class names, connect each to the image id
# classes = pd.read_csv(DATA_PATH + "image_class_labels.txt",' ',names=['image_id','cub_id'])
# classes = classes[classes["cub_id"].isin(cub_species)]
# images = pd.merge(cub,classes,on='image_id')
# # Read in the names of the image files, connect each to its image_id
# files = pd.read_csv(DATA_PATH + "images.txt",' ',names=['image_id','path'])
# images = pd.merge(images,files,on='image_id')
# # Fix the path
# images = pd.merge(images,birdlist,on="cub_id")
# images['path'] = images['folder_name'] + '/' + images['path'].str.split('/').str.get(1)
# images = images.drop(["image_id","cub_id","birdsnap_id","nabirds_id","folder_name"],axis=1)
# images = images.reindex(["path","left","upper","right","lower","id"],axis=1)
# images
# images.to_csv("C:/datasets/Combined/cub_ready.txt", header=True, index=False)

In [191]:
DIR="C:/datasets/Combined/"

In [189]:
# Done! Creating a master file
# file = pd.read_csv(DIR + "cub_ready.txt")
# file.to_csv(DIR + "all_ready.txt", header=True, index=False)   # Only write the header the first time
# file = pd.read_csv(DIR + "birdsnap_ready.txt")
# file.to_csv(DIR + "all_ready.txt", header=False, index=False, mode='a')
# file = pd.read_csv(DIR + "nabirds_ready.txt")
# file.to_csv(DIR + "all_ready.txt", header=False, index=False, mode='a')

In [196]:
# # Done! Split the file
# # train test split 50/50
# filename = DIR + "all_train.txt"
# allbirds = pd.read_csv(DIR + "all_ready.txt")
# sample = allbirds[allbirds["id"] == 0].sample(frac=0.5)
# sample.to_csv(filename, header=True, index=False)
# for i in np.arange(1,30):
#     sample = allbirds[allbirds["id"] == i].sample(frac=0.5)
#     sample.to_csv(filename, header=False, index=False, mode='a')

In [201]:
# # Determine the test set based on the training set
# all_data = pd.read_csv(DIR + "all_ready.txt")
# all_train = pd.read_csv(DIR + "all_train.txt")
# train_paths = list(all_train["path"])
# all_test = all_data[all_data["path"].isin(train_paths)]
# all_test.to_csv(DIR + "all_test.txt", header=True, index=False)

In [207]:
train = pd.read_csv(DIR + "all_train.txt")
test = pd.read_csv(DIR + "all_test.txt")

def finish(frame, folder):
    frame2 = frame.copy()
    frame2['path'] = frame2['path'].map(lambda x: os.path.dirname(x) + '/2_' + os.path.basename(x))
    frame = pd.concat([frame, frame2])
    frame['file'] = frame['path']
    frame['path'] = frame['path'].map(lambda x: folder + x)
    return frame

finish(train, 'train/')

,path,left,upper,right,lower,id,file
0,train/American_Goldfinch/f7f10a95-0dc5-446d-83...,107.0,153.0,495.0,543.0,0,American_Goldfinch/f7f10a95-0dc5-446d-8328-371...
1,train/American_Goldfinch/American_Goldfinch_00...,183.0,182.0,330.0,358.0,0,American_Goldfinch/American_Goldfinch_0032_319...
2,train/American_Goldfinch/247199.jpg,309.0,170.0,1365.0,990.0,0,American_Goldfinch/247199.jpg
3,train/American_Goldfinch/American_Goldfinch_00...,102.0,71.0,384.0,316.0,0,American_Goldfinch/American_Goldfinch_0091_319...
4,train/American_Goldfinch/74b43483-8f86-4c02-a4...,108.0,188.0,654.0,665.0,0,American_Goldfinch/74b43483-8f86-4c02-a406-5b1...
5,train/American_Goldfinch/247079.jpg,393.0,601.0,1626.0,1326.0,0,American_Goldfinch/247079.jpg
6,train/American_Goldfinch/246759.jpg,1022.0,701.0,1529.0,1108.0,0,American_Goldfinch/246759.jpg
7,train/American_Goldfinch/American_Goldfinch_01...,115.0,153.0,305.0,447.0,0,American_Goldfinch/American_Goldfinch_0100_321...
8,train/American_Goldfinch/b9d3c1a9-a55e-44db-8a...,57.0,223.0,337.0,539.0,0,American_Goldfinch/b9d3c1a9-a55e-44db-8afe-dd5...
9,train/American_Goldfinch/247567.jpg,426.0,249.0,772.0,870.0,0,American_Goldfinch/247567.jpg
